# 5. Live coding

This notebook will show you how we analyse a text in real life. To do so, we will examine two judge responses to asylum's claims in the UK.







Legend of symbols:

- 🤓: Tips

- 🤖📝: Your turn

- ❓: Question

- 💫: Extra exercise 

## 5.1. Read text

As we have learned in this course, the first step is to import the text into this notebook.

Two approaches:

- 1) Copy and paste content in a **<tt>.txt<tt>** file.
- 2) Install **<tt>pdftotext<tt>**: https://github.com/jalan/pdftotext.

In [ ]:
# 1)
# Read the raw file from txt
f = open('../data/asylum_claims.txt','r')
text = f.read()
f.close()

In [ ]:
text

In [ ]:
# Let's substitute \n by spaces
import re
text = re.sub('\n', ' ', text)

In [ ]:
print(text)

In [ ]:
import pdftotext

# Load your PDF
with open("../data/PA059452018.pdf", "rb") as f:
    pdf = pdftotext.PDF(f)

# If it's password-protected
with open("../data/PA002402019.pdf", "rb") as f:
    pdf = pdftotext.PDF(f, "secret")

# How many pages?
print(len(pdf))

# Iterate over all the pages
for page in pdf:
    print(page)

# Read some individual pages
print(pdf[0])
print(pdf[1])

# Read all the text into one string
pdf = "\n\n".join(pdf)
print(pdf)

## 5.2. Basic statistics

🤓 It is important when analysing text to know the basic figures: 
- How many words do we have? 
- How many sentences? 
- What are the most common words? 


❓ More questions?

### 5.2.1. How many words do we have? 

In [ ]:
# How many words do we have?
words_txt = text.split()
len(words_txt)

In [ ]:
words_pdf = pdf.split()
len(words_pdf)

### 5.2.2. How many sentences do we have? 

In [ ]:
# How many sentences do we have?
sent_txt = text.split('.')
len(sent_txt)

In [ ]:
# How many sentences do we have?
sent_pdf = pdf.split('.')
len(sent_pdf)

In [ ]:
# How many sentences do we have?
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize

sent_txt_nltk = sent_tokenize(text)
print(len(sent_txt_nltk))

In [ ]:
# How many sentences do we have?
sent_pdf_nltk = sent_tokenize(pdf)
print(len(sent_pdf_nltk))

### 5.2.3. What are the most common words? 

In [ ]:
# What are the most common words? 
wordfreq_txt = []

# count words in text
for w in words_txt:
    wordfreq_txt.append(words_txt.count(w))
    
# create a list with words and its frequency
word_list = list(set(zip(words_txt, wordfreq_txt)))

    
print("Pairs\n" + str(word_list))

In [ ]:
# What are the most common words? 
wordfreq_pdf = []

# count words in text
for w in words_pdf:
    wordfreq_pdf.append(words_pdf.count(w))
    
# create a list with words and its frequency
word_list = list(set(zip(words_pdf, wordfreq_pdf)))

# function to sort the list by second item of tuple
def sort_pairs(tup): 
  
    # reverse = None (Sorts in Ascending order) 
    # key is set to sort using second element of 
    # sublist lambda has been used 
    return(sorted(tup, key = lambda x: x[1], reverse = True))  

word_list_sort = sort_pairs(word_list)
    
print("Pairs\n" + str(word_list_sort))

In [ ]:
import pandas as pd

df = pd.DataFrame(word_list_sort)
df.columns = ['words', 'counts']

In [ ]:
df.head()

In [ ]:
import matplotlib.pyplot as plt
import plotly.express as px

fig = px.bar(df.loc[0:10,:], x='counts', y='words', orientation='h', text = 'words',
             labels={
                     "counts": "Frequency",
                     "words": "Words"
                 },)
fig.layout.yaxis.type = 'category'
fig.update_layout(yaxis_categoryorder = 'total ascending')
fig.update_layout(yaxis=dict(showticklabels=False))
fig.update_traces(texttemplate='%{text}', textposition='auto', marker_color='green')
fig.update_layout(uniformtext_minsize=8, uniformtext_mode='hide',   title={
        'text': "Words Frequency in Tribunal Appeals",
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})
fig.update_layout(
    autosize=False,
    width=1050,
    height=500)
fig.show()

❓ Does that give information of the content?

## 5.3. Clean text

We now clean the text with some  techniques we have learned.

### 5.3.1. Lowercase

In [ ]:
# Remove capital letters
text_clean = ' '.join(w.lower() for w in text.split())

In [ ]:
text_clean

### 5.3.2. Stop words 

In [ ]:
# Remove stopwords
from nltk.corpus import stopwords
stopwords = stopwords.words('english')

In [ ]:
text_clean = ' '.join(w for w in text_clean.split() if w not in stopwords)

In [ ]:
text_clean

### 5.3.3. Lemmatization

In [ ]:
# Remove puncutaction symbols
text_clean = [[token.lemma_ for token in sentence] for sentence in nlp(text_clean).sents]

In [ ]:
text_clean

### 5.3.4. Count words

In [ ]:
from collections import Counter

text_clean_counter = dict(Counter(text_clean))

In [ ]:
df_clean = pd.DataFrame.from_dict(text_clean_counter, orient='index')

In [ ]:
df_clean.reset_index(level=0, inplace=True)
df_clean.columns = ['words', 'counts']

In [ ]:
df_clean

In [ ]:
fig = px.bar(df_clean.loc[0:10,:], x='counts', y='words', orientation='h', text = 'words',
             labels={
                     "counts": "Frequency",
                     "words": "Words"
                 },)
fig.layout.yaxis.type = 'category'
fig.update_layout(yaxis_categoryorder = 'total ascending')
fig.update_layout(yaxis=dict(showticklabels=False))
fig.update_traces(texttemplate='%{text}', textposition='auto', marker_color='purple')
fig.update_layout(uniformtext_minsize=8, uniformtext_mode='hide',   title={
        'text': "Words Frequency in Tribunal Appeals",
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})
fig.update_layout(
    autosize=False,
    width=1050,
    height=500)
fig.show()

## 5.4. Word cloud

Now, let's show the word cloud of this text:

In [ ]:
import matplotlib.pyplot as plt
from wordcloud import WordCloud

word_cloud = WordCloud(background_color="white", repeat=True)
word_cloud.generate(text)

plt.axis("off")
plt.imshow(word_cloud, interpolation="bilinear")
plt.show()

## 5.5. What we have learned?

### 🤖📝 Now it's your turn:

🤖📝 Find the word 'EURODAC' using the function **<tt>search<tt>** from the **<tt>re<tt>** package.

🤖📝 Create a word cloud with different colour pattern using the text from the PDF.

🤖📝 Remove symbols from **<tt>df_clean<tt>** and plot again the frequency of words.